In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Inspiração
Algumas das questões interessantes (tarefas) que podem ser realizadas neste conjunto de dados -

Compreender qual conteúdo está disponível em diferentes países

Identificar conteúdo semelhante combinando recursos baseados em texto

Análise de rede de atores / diretores e encontrar insights interessantes

A Netflix tem se concentrado cada vez mais na TV ao invés de filmes nos últimos anos.

In [ ]:
df=pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')

vamos agora google as categorias e explorá-las

TV-MA: Este programa foi projetado especificamente para ser visto por adultos e, portanto, pode não ser adequado para crianças menores de 17 anos.

TV-14: Este programa contém algum material que muitos pais considerariam impróprio para crianças menores de 14 anos.

TV-PG: Este programa contém material que os pais podem considerar impróprio para crianças pequenas.

R: Menores de 17 anos exigem o acompanhamento dos pais ou responsável adulto. Os pais são incentivados a aprender mais sobre o filme antes de levar seus filhos pequenos com eles.

PG-13: Alguns materiais podem ser inadequados para crianças menores de 13 anos. Os pais devem ser cautelosos. Alguns materiais podem ser inadequados para pré-adolescentes.

NR ou UR: se um filme não foi enviado para classificação ou é uma versão não cortada de um filme que foi enviado

PG: Alguns materiais podem não ser adequados para crianças. Pode conter alguns materiais que os pais podem não gostar para seus filhos pequenos.

TV-Y7: Este programa foi criado para crianças a partir de 7 anos.

TV-G: Este programa é adequado para todas as idades.

TV-Y: os programas classificados como TV-Y são projetados para serem apropriados para crianças de todas as idades. Os elementos temáticos retratados em programas com esta classificação são projetados especificamente para um público muito jovem, incluindo crianças de 2 a 6 anos.

TV-Y7-FV: recomendado para maiores de 7 anos, com o único aviso de que o programa contém violência fantasiosa.

G: Todas as idades admitidas. Nada que ofenderia os pais por ver pelos filhos.

NC-17: Ninguém com 17 anos e menos admitidos. Claramente adulto. Crianças não são admitidas.
aqui descobrimos que UR e NR têm a mesma classificação (sem classificação, sem classificação)
Versões sem cortes / estendidas de filmes rotulados como "Sem cortes" também contêm avisos dizendo que a versão sem cortes do filme possui conteúdo diferente do lançamento nos cinemas e pode não ser adequada para menores.
então temos a solução para isso.

In [ ]:
df['country'].replace(np.nan, 'United States',inplace  = True)
df = df.dropna(subset=['rating','date_added'])
df.isnull().sum() 

In [ ]:
df["date_added"] = pd.to_datetime(df['date_added'])
df['day_added'] = df['date_added'].dt.day
df['year_added'] = df['date_added'].dt.year
df['month_added']=df['date_added'].dt.month
df['week_added']=df['date_added'].dt.weekday
df['year_added'].astype(int);
df['day_added'].astype(int);

In [ ]:
for i in df.index:
    if df.loc[i,'rating']=='UR':
        df.loc[i,'rating']='NR'

In [ ]:
df.head(5)

In [ ]:
plt.figure(figsize=(18,6))
df['rating'].value_counts(normalize=True).plot.bar()
plt.title('Distribuição por categoria')
plt.xlabel('Categoria')
plt.ylabel('Frequência')
plt.show()

Em 2019, os EUA vêem produzindo mais filmes para adolecentes com o concentimento de adultos.

In [ ]:
top_productive_EUA=df[(df['country']=='United States') & (df.year_added >= 2016)]

plt.figure(figsize=(18,8))
sns.countplot(x='year_added',hue='rating',data=top_productive_EUA)
plt.title('Comparando as categorias nos EUA após 2016')
plt.show()

A India, em 2019, produziu mais conteudos para adultos comparado com os anos anteriores.

In [ ]:
top_productive_EUA=df[(df['country']=='India') & (df.year_added >= 2016)]

plt.figure(figsize=(18,8))
sns.countplot(x='year_added',hue='rating',data=top_productive_EUA)
plt.title('Comparando as categorias nos India após 2016')
plt.show()

In [ ]:
df.rating.unique()

In [ ]:
top_productive_countries=df[(df['country']=='United States')|(df['country']=='India')|(df['country']=='United Kingdom')|(df['country']=='Japan')|
                             (df['country']=='Canada')|(df['country']=='Spain')]
plt.figure(figsize=(10,8))
sns.countplot(x='country',hue='type',data=top_productive_countries)
plt.title('comparing between the types that the top countries produce')
plt.show()

In [ ]:
top_productive_EUA=df[(df['country']=='United States')]

plt.figure(figsize=(10,8))
sns.countplot(x='year_added',hue='type',data=top_productive_EUA)
plt.title('Comparando o tipo de produção na EUA')
plt.show()

In [ ]:
top_productive_EUA=df[(df['country']=='India')]

plt.figure(figsize=(10,8))
sns.countplot(x='year_added',hue='type',data=top_productive_EUA)
plt.title('Comparando o tipo de produção na India')
plt.show()

In [ ]:
top_productive_EUA=df[(df['country']=='United Kingdom')]

plt.figure(figsize=(10,8))
sns.countplot(x='year_added',hue='type',data=top_productive_EUA)
plt.title('Comparando o tipo de produção na Inglaterra')
plt.show()

In [ ]:
df.columns

In [ ]:
df.type.unique()

In [ ]:
paises = ['United States', 'India', 'United Kingdom', 'Japan', 'Canada',
       'South Korea', 'Spain', 'France', 'Mexico', 'Turkey', 'Australia',
       'Taiwan', 'Hong Kong', 'United Kingdom, United States', 'Egypt']

In [ ]:
country_count = country_count
fig=plt.figure(figsize=(20,10))
sns.barplot(y=country_count.index[:20],x=country_count.values[:20] )

In [ ]:
from collections import Counter
genre = df['listed_in']
genre_count = pd.Series(dict(Counter(','.join(genre).replace(' ,',',').replace(', ',',')
                                       .split(',')))).sort_values(ascending=False)

plt.figure(figsize=(18,6))
genre_count.plot.bar()

In [ ]:
plt.subplots(figsize=(25,15))
wordcloud = WordCloud(
                          background_color='Black',
                          width=1920,
                          height=1080
                         ).generate(" ".join(df.listed_in))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()


In [ ]:
plt.subplots(figsize=(25,15))
wordcloud = WordCloud(
                          background_color='Black',
                          width=1920,
                          height=1080
                         ).generate(" ".join(df.title))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
categories = ", ".join(df.cast.fillna("")).split(",")
cast=pd.Series(categories)

In [ ]:
plt.subplots(figsize=(25,15))
wordcloud = WordCloud(
                          background_color='Black',
                          width=1920,
                          height=1080
                         ).generate(" ".join(cast))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
plt.subplots(figsize=(25,15))
wordcloud = WordCloud(
                          background_color='Black',
                          width=1920,
                          height=1080
                         ).generate(" ".join(director))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
import plotly.graph_objects as go

def country_top_dir(country):
    dir=df.loc[(df.country==str(country)) & (df.type=="Movie")]


    col = "director"
    categories = ", ".join(dir[col].fillna("")).split(", ")

    directors=pd.Series(categories)
    directors=directors.value_counts()[1:16]

    trace=go.Bar(x=directors.values[:10][::-1],y=directors.index[:10][::-1],orientation='h',marker=dict(color='#a678de'))
    return trace
from plotly.subplots import make_subplots
traces = []
titles = ["United States", "","India","", "United Kingdom", "Canada","", "Spain","", "Japan","France"]
for title in titles:
    if title != "":
        traces.append(country_top_dir(title))

fig = make_subplots(rows=3, cols=5, subplot_titles=titles)
fig.add_trace(traces[0], 1,1)
fig.add_trace(traces[1], 1,3)
fig.add_trace(traces[2], 1,5)
fig.add_trace(traces[3], 2,1)
fig.add_trace(traces[4], 2,3)
fig.add_trace(traces[5], 2,5)
fig.add_trace(traces[6], 3,1)

fig.update_layout(height=1200, showlegend=False)
fig.show()